In [1]:
!pip install -U tensorflow

    100% |████████████████████████████████| 58.4MB 604kB/s ta 0:00:013  16% |█████▎                          | 9.6MB 2.5MB/s eta 0:00:20    21% |███████                         | 12.7MB 3.3MB/s eta 0:00:14    72% |███████████████████████▏        | 42.2MB 1.8MB/s eta 0:00:10    95% |██████████████████████████████▌ | 55.7MB 2.2MB/s eta 0:00:02
    100% |████████████████████████████████| 12.2MB 1.4MB/s ta 0:00:01   33% |██████████▋                     | 4.0MB 3.2MB/s eta 0:00:03
    100% |████████████████████████████████| 3.3MB 4.0MB/s ta 0:00:01
  Found existing installation: numpy 1.15.0
    Uninstalling numpy-1.15.0:
      Successfully uninstalled numpy-1.15.0
  Found existing installation: tensorboard 1.9.0
    Uninstalling tensorboard-1.9.0:
      Successfully uninstalled tensorboard-1.9.0
  Found existing installation: tensorflow 1.9.0
    Uninstalling tensorflow-1.9.0:
      Successfully uninstalled tensorflow-1.9.0


In [2]:
import tensorflow as tf

/home/sid/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
from __future__ import print_function
import shutil
import os.path
import matplotlib.pyplot as plt
from IPython.display import display, Image
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [4]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


# Setting up Parameters

This will our tuning knobs for our MNIST Model

In [5]:
learning_rate = 0.001
training_iters = 20000
batch_size = 128
display_step = 10

# Network parameters

In [6]:
n_input = 28*28
n_classes = 10
dropout = 0.75

# tf graph input

In [7]:
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32) #dropout(keep probability)

# Create layer wrapper

In [8]:
def conv2d(x, w, b, strides=1):
    #Conv2d wrapper, with bias and relu activation
    x = tf.nn.conv2d(x, w, strides=[1,strides,strides,1],padding='SAME')
    x = tf.nn.bias_add(x,b)
    return tf.nn.relu(x)

In [9]:
def maxpool2d(x, k=2):
    #MaxPool2d wrapper
    return tf.nn.max_pool(x, ksize=[1,k,k,1], strides=[1,k,k,1],padding='SAME')

# Store layers weight & bias

In [10]:
weights = {
    #5x5 conv, 1 input, 32  outputs
    'wc1': tf.Variable(tf.random_normal([5,5,1,32])),
    #5x5 conv, 32 inputs, 64 outputs
    'wc2': tf.Variable(tf.random_normal([5,5,32,64])),
    #fully connected, 7*7*64 inputs, 1024 outputs
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    #1024 inputs, 10 outputs (class prediction)
    'out': tf.Variable(tf.random_normal([1024, n_classes]))
}

In [11]:
biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# Creating our Sequential Neural Network

In [12]:
def conv_net(x, weights, biases, dropout):
    #Reshape input picture
    x = tf.reshape(x, shape=[-1,28,28,1])
    
    #Convolution Layer
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    #Max Pooling
    conv1 = maxpool2d(conv1, k=2)
    
    #Convolution Layer
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    #Max Pooling
    conv2 = maxpool2d(conv2, k=2)
    
    #Fully connected Layer
    #Reshape conv2 output to fit fully connected layer input
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    #Apply Dropout
    fc1 = tf.nn.dropout(fc1, dropout)
    
    #Output, class prediction
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

In [13]:
EXPORT_DIR = '/home/sid/Desktop/tensorflow/'

# Construct model

In [ ]:
pred = conv_net(x, weights, biases, keep_prob)

#Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

#Evaluate model
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

#Initializing the variables
init = tf.initialize_all_variables()

#Launch the graph
with tf.Session() as sess:
    sess.run(init)
    step = 1
    # Keep training until reach max iterations
    while step * batch_size < training_iters:
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Run optimization op (backprop)
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y,
                                       keep_prob: dropout})
        if step % display_step == 0:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([cost, accuracy], feed_dict={x: batch_x,
                                                              y: batch_y,
                                                              keep_prob: 1.})
            print("Iter " + str(step * batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc))
        step += 1
    print("Optimization Finished!")

    # Calculate accuracy for 256 mnist test images
    print("Testing Accuracy:", \
          sess.run(accuracy, feed_dict={x: mnist.test.images[:256],
                                        y: mnist.test.labels[:256],
                                        keep_prob: 1.}))
    WC1 = weights['wc1'].eval(sess)
    BC1 = biases['bc1'].eval(sess)
    WC2 = weights['wc2'].eval(sess)
    BC2 = biases['bc2'].eval(sess)
    WD1 = weights['wd1'].eval(sess)
    BD1 = biases['bd1'].eval(sess)
    W_OUT = weights['out'].eval(sess)
    B_OUT = biases['out'].eval(sess)

# Create new graph for exporting
g = tf.Graph()
with g.as_default():
    x_2 = tf.placeholder("float", shape=[None, 784], name="input")

    WC1 = tf.constant(WC1, name="WC1")
    BC1 = tf.constant(BC1, name="BC1")
    x_image = tf.reshape(x_2, [-1, 28, 28, 1])
    CONV1 = conv2d(x_image, WC1, BC1)
    MAXPOOL1 = maxpool2d(CONV1, k=2)

    WC2 = tf.constant(WC2, name="WC2")
    BC2 = tf.constant(BC2, name="BC2")
    CONV2 = conv2d(MAXPOOL1, WC2, BC2)
    MAXPOOL2 = maxpool2d(CONV2, k=2)

    WD1 = tf.constant(WD1, name="WD1")
    BD1 = tf.constant(BD1, name="BD1")

    FC1 = tf.reshape(MAXPOOL2, [-1, WD1.get_shape().as_list()[0]])
    FC1 = tf.add(tf.matmul(FC1, WD1), BD1)
    FC1 = tf.nn.relu(FC1)

    W_OUT = tf.constant(W_OUT, name="W_OUT")
    B_OUT = tf.constant(B_OUT, name="B_OUT")

    # skipped dropout for exported graph as there 
    # is no need for already calculated weights

    OUTPUT = tf.nn.softmax(tf.matmul(FC1, W_OUT) + B_OUT, name="output")

    sess = tf.Session()
    init = tf.initialize_all_variables()
    sess.run(init)

    graph_def = g.as_graph_def()
    tf.train.write_graph(graph_def, EXPORT_DIR, 'mnist_model_graph.pb', as_text=False)

    # Test trained model
    y_train = tf.placeholder("float", [None, 10])
    correct_prediction = tf.equal(tf.argmax(OUTPUT, 1), tf.argmax(y_train, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

    print("check accuracy %g" % accuracy.eval(
            {x_2: mnist.test.images, y_train: mnist.test.labels}, sess))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Iter 1280, Minibatch Loss= 26427.568359, Training Accuracy= 0.29688
Iter 2560, Minibatch Loss= 11429.310547, Training Accuracy= 0.53906
Iter 3840, Minibatch Loss= 5434.896973, Training Accuracy= 0.66406
Iter 5120, Minibatch Loss= 2484.625977, Training Accuracy= 0.80469
Iter 6400, Minibatch Loss= 2558.153809, Training Accuracy= 0.80469
Iter 7680, Minibatch Loss= 3869.490723, Training Accuracy= 0.78906
Iter 8960, Minibatch Loss= 3164.463135, Training Accuracy= 0.82031
Iter 10240, Minibatch Loss= 1616.154297, Training Accuracy= 0.89844
Iter 11520, Minibatch Loss= 2798.760010, Training Accuracy= 0.80469
Iter 12800, Minibatch Loss= 1887.491211, Training Accuracy= 0.88281
Iter 14080, Minibatch Loss= 2861.651855, Training Accuracy=

# Training of the model finished with testing accuracy of 93.75%